In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the trips csv as a dataframe 
df = pd.read_csv("trips.csv")

# remove unnessecary rows
df.drop(["route_desc", "route_long", "route_type", "service_id", "route_colo", "route_text", 
         "wheelchair", "UID", "O_AREA", "D_AREA", "TRIP_ASOD", "VEH_ID", "AC", "DIGI_DATE", "DIGI_SPE", 
         "DIGI_NOTES", "ASSIGN_ID", "H3RES5_IDX", "RANKING", "INST_NUM", "FARE_Q219"], axis=1, inplace= True)
df.head()

,route_id,route_shor,trip_id,trip_heads,trip_short,direction_,agency_id,headway_se,O_TERMINAL,LEN_KM,STD_FARE
0,P_O_14_CA165,CA165,P_O_14_CA165_O,Roxy,Mazallat-Roxy,0,P_O_14,1367.0,Mazallat,9.504617,3.0
1,CTA_916,916,CTA_916_R,Amiriya,Moneeb-Amiriya,1,CTA,3844.0,Moneeb,26.536089,3.0
2,P_O_14_GZ041,GZ041,P_O_14_GZ041_R,26th of July (Ring Road),Moneeb-26th of July (Ring Road),1,P_O_14,192.0,Moneeb,21.900026,4.0
3,CTA_975,975,CTA_975_R,Hegaz Square,Moassasa-Hegaz Square,1,CTA,2397.0,Moassasa,21.800161,3.0
4,P_O_14_CA170,CA170,P_O_14_CA170_O,10th of Ramadan Station (Asher Men Ramadan),Manshiyat Nasser-10th of Ramadan Station (Ashe...,0,P_O_14,2797.0,Manshiyat Nasser,30.455949,5.0


### Subset Data - Keep only Cairo Transport Authority (CTA) routes

In [3]:
bool_series = df['agency_id'] == 'CTA'  #true for all rows that have agency_id = CTA
df_CTA = df[bool_series]

In [4]:
# sort by route number (to check if trips in same route have same headway : TRUE)
# reset index with drop = true so that a new column isn't added: https://stackoverflow.com/questions/16167829/in-pandas-how-can-i-reset-index-without-adding-a-new-column
df_CTA = df_CTA.sort_values('route_shor', ascending= 1).reset_index(drop=True)


### Get number of Buses: 

#### 1) trip duration

In [5]:
# get number of buses
# assume a constant speed of 15km/h across network (4.2m/s)
# multiply by 2 to get the time for the whole route
df_CTA['route_time_s'] = ((df_CTA["LEN_KM"] *1000) / 4.2) * 2
df_CTA.head()

,route_id,route_shor,trip_id,trip_heads,trip_short,direction_,agency_id,headway_se,O_TERMINAL,LEN_KM,STD_FARE,route_time_s
0,CTA_1-1140,1-1140,CTA_1-1140_R,Imbaba,Moneeb-Imbaba,1,CTA,6688.0,Moneeb,24.034205,3.0,11444.859596
1,CTA_1-1140,1-1140,CTA_1-1140_O,Moneeb,Imbaba-Moneeb,0,CTA,6688.0,Imbaba,19.926860,3.0,9488.980942
2,CTA_1002,1002,CTA_1002_R,Matbaa,Masaken Ain Shams-Matbaa,1,CTA,1644.0,Masaken Ain Shams,41.860953,4.0,19933.786928
3,CTA_1002,1002,CTA_1002_O,Masaken Ain Shams,Matbaa-Masaken Ain Shams,0,CTA,1644.0,Matbaa,37.057157,4.0,17646.265073
4,CTA_1003,1003,CTA_1003_O,Bulaq Al Dakrur,Cairo Airport-Bulaq Al Dakrur,0,CTA,4270.0,Cairo Airport,41.049288,3.0,19547.279850


#### 2) no. of buses

In [6]:
# get no. of buses for each trip (route time/ headway)/2
# divide by 2 because this is the number of buses for the whole route (the rows are trips: each route has two trips)
df_CTA['buses'] = ((df_CTA["route_time_s"]/df_CTA["headway_se"]) / 2)
df_CTA.head()

,route_id,route_shor,trip_id,trip_heads,trip_short,direction_,agency_id,headway_se,O_TERMINAL,LEN_KM,STD_FARE,route_time_s,buses
0,CTA_1-1140,1-1140,CTA_1-1140_R,Imbaba,Moneeb-Imbaba,1,CTA,6688.0,Moneeb,24.034205,3.0,11444.859596,0.855626
1,CTA_1-1140,1-1140,CTA_1-1140_O,Moneeb,Imbaba-Moneeb,0,CTA,6688.0,Imbaba,19.926860,3.0,9488.980942,0.709403
2,CTA_1002,1002,CTA_1002_R,Matbaa,Masaken Ain Shams-Matbaa,1,CTA,1644.0,Masaken Ain Shams,41.860953,4.0,19933.786928,6.062587
3,CTA_1002,1002,CTA_1002_O,Masaken Ain Shams,Matbaa-Masaken Ain Shams,0,CTA,1644.0,Matbaa,37.057157,4.0,17646.265073,5.366869
4,CTA_1003,1003,CTA_1003_O,Bulaq Al Dakrur,Cairo Airport-Bulaq Al Dakrur,0,CTA,4270.0,Cairo Airport,41.049288,3.0,19547.279850,2.288909


In [7]:
# get sum of buses to compare to reported total buses of 2650
df_CTA["buses"].sum()

560.3277221549987

### get ratio of calculated sum to reported sum

In [8]:
total_buses = 2650
ratio = total_buses/df_CTA["buses"].sum()   # 18.69


### Multiply bus column by ratio to reach figure closer to 2650

In [9]:
# .apply(np.ceil) to round values up (we don't want any trips with 0 values)
df_CTA['buses'] = (df_CTA['buses']*ratio).apply(np.ceil)
##df_CTA.head()

In [10]:
# we need the number of buses for each route to be split equally between trips
# the number of buses for each trip was calculated seperately based on the trip length (varies between outbound and return)
# this groups by route, and assigns both trips the lowest of the two 'buses' value calculated
# https://stackoverflow.com/questions/49591867/create-new-column-with-max-value-with-groupby
df_CTA['standard_buses'] = df_CTA.groupby(['route_id'])['buses'].transform(min)
df_CTA.head()

,route_id,route_shor,trip_id,trip_heads,trip_short,direction_,agency_id,headway_se,O_TERMINAL,LEN_KM,STD_FARE,route_time_s,buses,standard_buses
0,CTA_1-1140,1-1140,CTA_1-1140_R,Imbaba,Moneeb-Imbaba,1,CTA,6688.0,Moneeb,24.034205,3.0,11444.859596,5.0,4.0
1,CTA_1-1140,1-1140,CTA_1-1140_O,Moneeb,Imbaba-Moneeb,0,CTA,6688.0,Imbaba,19.926860,3.0,9488.980942,4.0,4.0
2,CTA_1002,1002,CTA_1002_R,Matbaa,Masaken Ain Shams-Matbaa,1,CTA,1644.0,Masaken Ain Shams,41.860953,4.0,19933.786928,29.0,26.0
3,CTA_1002,1002,CTA_1002_O,Masaken Ain Shams,Matbaa-Masaken Ain Shams,0,CTA,1644.0,Matbaa,37.057157,4.0,17646.265073,26.0,26.0
4,CTA_1003,1003,CTA_1003_O,Bulaq Al Dakrur,Cairo Airport-Bulaq Al Dakrur,0,CTA,4270.0,Cairo Airport,41.049288,3.0,19547.279850,11.0,11.0


### see what the sum is now: it needs to be close to 2650

In [11]:
# see what the sum is now
df_CTA["standard_buses"].sum()

2742.0

In [12]:
df_CTA["standard_buses"].describe()

count    340.000000
mean       8.064706
std        8.710109
min        1.000000
25%        4.000000
50%        5.000000
75%        8.000000
max       74.000000
Name: standard_buses, dtype: float64

### save file to csv

In [13]:
df_CTA.to_csv(r'buses_per_trip.csv', index = None, header=True)